In [1]:
import numpy as np

In [2]:
x = np.load('data/features.npy')
y = np.load('data/true_cost.npy')
display(x.shape)
display(y.shape)

(27, 2500, 2866)

(27, 2500)

In [7]:
from solver import solve_bmatching
import torch 

sols = [torch.from_numpy(solve_bmatching(yi)) for yi in y]
sols = torch.stack(sols)
sols.shape


torch.Size([27, 2500])

In [12]:
sols.unique(dim=0).shape

torch.Size([27, 2500])

In [47]:
import gurobipy as gp 

def solve_matching_gurobi(preds):
    y = -preds
    N1 = np.zeros((50,2500))
    N2 = np.zeros_like(N1)
    b1 = np.ones(50)
    b2 = np.ones_like(b1)
    
    for i in range(50):
        rowmask = np.zeros((50,50))
        colmask = np.zeros_like(rowmask)
        rowmask[i,:] = 1 
        colmask[:,i] = 1
        N1[i] = rowmask.flatten()
        N2[i] = colmask.flatten() 
    G = np.vstack((N1, N2))
    h = np.concatenate((b1, b2))

    model = gp.Model('mathcing')
    model.setParam('OutputFlag',0)
    model.setParam('PoolSearchMode', 2)
    model.setParam('PoolSolutions', 10)
    model.setParam('PoolGap', 0.0)
    x = model.addMVar(shape=G.shape[1], vtype=gp.GRB.BINARY, name='x')
    model.setObjective(y @ x, gp.GRB.MINIMIZE)
    model.addConstr(G @ x <= h, name="ineq")
    model.optimize()
    return x.X, model.SolCount

In [48]:
counts = np.array([solve_matching_gurobi(yi)[1] for yi in y])
np.where(counts !=1 )

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26]),)

(array([0]),)

In [39]:
y[0] @ sg

41.0